# NIDIS Visualization Tool

Kernel: `ILAB Kernel (PyTorch)`

Purpose: To visualize the indicator rasters produced by the NIDIS code base.

In [ ]:
import sys
sys.path.append('../../nidis')

from nidis.model.Metadata import (
    DictofInitialToWord_Seasons,
    DictofNumNamePairs_Channels
)

import xarray as xr
import pathlib

import panel as pn
import panel.widgets as pnw
import hvplot.xarray
assert hvplot.xarray

pn.extension(loading_spinner='dots', template='bootstrap')
pn.param.ParamMethod.loading_indicator = True

In [ ]:
nc_dir_str: str = '/panfs/ccds02/nobackup/projects/ilab/projects/NIDIS/Outputs_fame'

nc_dir: pathlib.Path = pathlib.Path(nc_dir_str)

ncs_in_dir: list = sorted(nc_dir.glob('*.nc'))

indicator_dict: dict = {}

for nc_file_path in ncs_in_dir:
    subsplit = nc_file_path.stem.split('_')
    key = '_'.join(subsplit[6:-1])
    season = DictofInitialToWord_Seasons[subsplit[5]]
    if not key in indicator_dict:
        indicator_dict[key] = {}
    indicator_dict[key][season] = [str(nc_file_path)]

In [ ]:
visible = pnw.Select(options=[True, False],
                     value=True,
                     name='Raster Visible')

selector = pnw.NestedSelect(options=indicator_dict,
                            levels=['Indicator', 'Season', 'File'],
                            layout=pn.Row)

def display_selection(selector_values: dict):
    indicator = selector_values['Indicator']
    season = selector_values['Season']
    file_path = selector_values['File']
    sampleArray = xr.load_dataset(file_path)
    rasterLayer = sampleArray['FracI'].hvplot(
                geo=True,
                cmap='jet',
                rasterize=True,
                frame_width=1200,
                tiles='EsriImagery',
                visible=visible)
    yield rasterLayer

In [ ]:
pn.Column(pn.Row(visible, selector), pn.bind(display_selection, selector))